<a href="https://colab.research.google.com/github/JahedNaghipoor/Chest_X_ray_Corona/blob/main/Chest_X_ray_Corona.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/Machine Learning and Deep Learning/Chest"

In [ ]:
%cd /content/gdrive/My Drive/Machine Learning and Deep Learning/Chest

In [ ]:
#!kaggle datasets download -d praveengovi/coronahack-chest-xraydataset

In [ ]:
#!unzip \*.zip  && rm *.zip

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from PIL import Image, ImageOps 
import matplotlib.pyplot as plt
%matplotlib inline

import os

In [ ]:
dir = "./Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/"
train_dir = os.path.join(dir,"train/")
test_dir = os.path.join(dir,"test/")

In [ ]:
df = pd.read_csv('Chest_xray_Corona_Metadata.csv')
df.head()

In [ ]:
df.describe()

In [ ]:

plt.figure(fig_size=(8,8)) 
missing_values = df.isnull().sum()
missing_values.plot.bar()

In [ ]:
df.fillna('unknown', inplace=True)
df.isnull().sum()

In [ ]:
print("Total num of train Images are: ",str(len(os.listdir(train_dir))))
print()
print("Total num of test Images are: ",str(len(os.listdir(test_dir))))

In [ ]:
sns.countplot(x =  'Label_1_Virus_category', data = df)

In [ ]:
sns.countplot(x='Label_2_Virus_category', data=df)

In [ ]:
df['Label'] = df['Label'].replace({'Pnemonia':1,'Normal':0})

train_df = df[df.Dataset_type=='TRAIN']
test_df = df[df.Dataset_type=='TEST']

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
train_df.Label.value_counts().plot.bar()

In [ ]:
test_df.Label.value_counts().plot.bar()

In [ ]:
df_normal = df[df.Label==0]
df_pneumonia = df[df.Label==1]
df_pneumonia_covid = df_pneumonia[df_pneumonia.Label_2_Virus_category=='COVID-19']
normal_images = df_normal.X_ray_image_name.values.tolist()
pneumonia_covid_images = df_pneumonia_covid.X_ray_image_name.values.tolist()

In [ ]:
plt.figure(figsize=(12,12))
for index, file in enumerate(pneumonia_covid_images[:4]):
    file_path = train_dir + str(file)
    image = Image.open(file_path)
    image = ImageOps.grayscale(image)
    image.thumbnail((224,224))
    plt.subplot(2,2,index+1)
    plt.imshow(image)

plt.tight_layout()

In [ ]:
plt.figure(figsize=(12,12))
for index, file in enumerate(normal_images[:4]):
    file_path = train_dir+ str(file)
    image = Image.open(file_path)
    image = ImageOps.grayscale(image)
    image.thumbnail((224,224))
    plt.subplot(2,2,index+1)
    plt.imshow(image)

plt.tight_layout()

In [ ]:
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Dropout, BatchNormalization, Flatten, Dense, AvgPool2D,MaxPool2D
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import EarlyStopping

In [ ]:
IMG_W = 150
IMG_H = 150
CHANNELS = 3

INPUT_SHAPE = (IMG_W, IMG_H, CHANNELS)
EPOCHS = 5
BATCH_SIZE = 6

In [ ]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3, 3), strides=(1,1), input_shape=INPUT_SHAPE, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=32, kernel_size=(3, 3), strides=(1,1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters= 64, kernel_size= (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
    
model.add(Flatten())
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(rate=0.25))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    dir,
    target_size=(IMG_H, IMG_W),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    dir, 
    target_size=(IMG_H, IMG_W),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle= False,
    subset='validation')

history = model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // BATCH_SIZE,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // BATCH_SIZE,
    epochs = EPOCHS)

In [ ]:
history.history

In [ ]:
print("training accuracy", history.history['accuracy'][-1])
print("validation accuracy", history.history['val_accuracy'][-1])

In [ ]:
acc = pd.DataFrame(model.history.history)
acc[['accuracy','val_accuracy']].plot()

In [ ]:
'''label = validation_generator.classes
pred= model.predict(validation_generator)
predicted_class_indices=np.argmax(pred,axis=1)'''

In [ ]:
'''from sklearn.metrics import confusion_matrix

cf = confusion_matrix(predicted_class_indices,label)
sns.heatmap(cf, annot =True)'''